In [73]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import pickle
import torch.nn as nn
from torch.autograd import Variable


In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import pickle

### Load the data

path = 'C:/Users/User/OneDrive - NTNU/NTNU/Prosjekt oppgave NLP/dataset/'
# First we load the entire CSV file into an m x 3
D = torch.tensor(pd.read_csv(path + "linreg-multi-synthetic-2.csv", header=None).values, dtype=torch.float)

print(D)
# We extract all rows and the first 2 columns, and then transpose it
x_dataset = D[:, 0:2].t()

# We extract all rows and the last column, and transpose it
y_dataset = D[:, 2].t()

# And make a convenient variable to remember the number of input columns
n = 2


### Model definition ###

# First we define the trainable parameters A and b 
A = torch.randn((1, n), requires_grad=True)
b = torch.randn(1, requires_grad=True)

# Then we define the prediction model
def model(x_input):
    return A.mm(x_input) + b


### Loss function definition ###

def loss(y_predicted, y_target):
    return ((y_predicted - y_target)**2).sum()

### Training the model ###

# Setup the optimizer object, so it optimizes a and b.
optimizer = optim.Adam([A, b], lr=0.1)

# Main optimization loop
for t in range(2000):
    # Set the gradients to 0.
    optimizer.zero_grad()
    # Compute the current predicted y's from x_dataset
    y_predicted = model(x_dataset)
    # See how far off the prediction is
    current_loss = loss(y_predicted, y_dataset)
    # Compute the gradient of the loss with respect to A and b.
    current_loss.backward()
    # Update A and b accordingly.
    optimizer.step()
    print(f"t = {t}, loss = {current_loss}, A = {A.detach().numpy()}, b = {b.item()}")

tensor([[ 0.0000e+00,  0.0000e+00,  5.8537e-01],
        [ 2.5063e-02,  0.0000e+00,  7.2942e+00],
        [ 5.0125e-02,  0.0000e+00, -6.0677e-01],
        ...,
        [ 9.9499e+00,  1.0000e+01,  3.5968e+01],
        [ 9.9749e+00,  1.0000e+01,  4.4485e+01],
        [ 1.0000e+01,  1.0000e+01,  3.6397e+01]])
t = 0, loss = 79876968.0, A = [[-0.3819827   0.22281507]], b = 1.7270499467849731
t = 1, loss = 71843392.0, A = [[-0.28215355  0.32262346]], b = 1.8268699645996094
t = 2, loss = 64263928.0, A = [[-0.1826227   0.42209542]], b = 1.9263750314712524
t = 3, loss = 57145600.0, A = [[-0.08352844  0.5210723 ]], b = 2.0254180431365967
t = 4, loss = 50493840.0, A = [[0.01497986 0.61938095]], b = 2.1238393783569336
t = 5, loss = 44312264.0, A = [[0.1127414  0.71683294]], b = 2.221466302871704
t = 6, loss = 38602520.0, A = [[0.20958385 0.8132243 ]], b = 2.318112850189209
t = 7, loss = 33363978.0, A = [[0.30532327 0.9083352 ]], b = 2.4135799407958984
t = 8, loss = 28593602.0, A = [[0.3997645 1.00

t = 121, loss = 1453249.0, A = [[2.004265  1.3598626]], b = 3.611776113510132
t = 122, loss = 1452961.75, A = [[2.0061035 1.3588221]], b = 3.6132729053497314
t = 123, loss = 1452707.375, A = [[2.0078485 1.3577975]], b = 3.614729166030884
t = 124, loss = 1452482.5, A = [[2.0094957 1.3567806]], b = 3.6161389350891113
t = 125, loss = 1452283.375, A = [[2.0110426 1.3557651]], b = 3.617497444152832
t = 126, loss = 1452106.375, A = [[2.0124874 1.3547459]], b = 3.6188015937805176
t = 127, loss = 1451947.0, A = [[2.0138302 1.3537196]], b = 3.6200497150421143
t = 128, loss = 1451802.25, A = [[2.015072  1.3526844]], b = 3.621241569519043
t = 129, loss = 1451669.375, A = [[2.0162156 1.3516397]], b = 3.62237811088562
t = 130, loss = 1451546.375, A = [[2.017264  1.3505864]], b = 3.6234614849090576
t = 131, loss = 1451430.75, A = [[2.0182214 1.3495262]], b = 3.624494791030884
t = 132, loss = 1451321.75, A = [[2.019093  1.3484617]], b = 3.6254818439483643
t = 133, loss = 1451218.625, A = [[2.0198843 

t = 241, loss = 1449135.375, A = [[2.0249093 1.3212606]], b = 3.7280609607696533
t = 242, loss = 1449126.0, A = [[2.024828  1.3211865]], b = 3.7289645671844482
t = 243, loss = 1449116.75, A = [[2.0247467 1.3211123]], b = 3.7298665046691895
t = 244, loss = 1449107.5, A = [[2.0246656 1.3210378]], b = 3.730766773223877
t = 245, loss = 1449098.625, A = [[2.0245848 1.320963 ]], b = 3.7316653728485107
t = 246, loss = 1449089.5, A = [[2.024504 1.320888]], b = 3.732562303543091
t = 247, loss = 1449080.5, A = [[2.0244234 1.3208128]], b = 3.733457565307617
t = 248, loss = 1449071.5, A = [[2.024343  1.3207375]], b = 3.73435115814209
t = 249, loss = 1449062.75, A = [[2.0242627 1.320662 ]], b = 3.735243082046509
t = 250, loss = 1449054.0, A = [[2.0241826 1.3205864]], b = 3.736133337020874
t = 251, loss = 1449045.125, A = [[2.0241027 1.3205107]], b = 3.7370219230651855
t = 252, loss = 1449036.25, A = [[2.024023 1.320435]], b = 3.7379088401794434
t = 253, loss = 1449027.75, A = [[2.0239437 1.3203593]

t = 357, loss = 1448383.875, A = [[2.016833  1.3133314]], b = 3.8202309608459473
t = 358, loss = 1448379.875, A = [[2.0167747 1.3132737]], b = 3.8208999633789062
t = 359, loss = 1448376.0, A = [[2.0167165 1.3132161]], b = 3.8215668201446533
t = 360, loss = 1448372.0, A = [[2.0166585 1.3131588]], b = 3.8222312927246094
t = 361, loss = 1448368.125, A = [[2.0166008 1.3131015]], b = 3.8228936195373535
t = 362, loss = 1448364.125, A = [[2.0165431 1.3130445]], b = 3.8235535621643066
t = 363, loss = 1448360.5, A = [[2.0164857 1.3129878]], b = 3.824211359024048
t = 364, loss = 1448356.5, A = [[2.0164285 1.3129312]], b = 3.824866771697998
t = 365, loss = 1448352.75, A = [[2.0163715 1.3128748]], b = 3.8255198001861572
t = 366, loss = 1448349.0, A = [[2.0163147 1.3128186]], b = 3.8261706829071045
t = 367, loss = 1448345.25, A = [[2.0162582 1.3127627]], b = 3.8268191814422607
t = 368, loss = 1448341.75, A = [[2.0162017 1.312707 ]], b = 3.827465534210205
t = 369, loss = 1448338.0, A = [[2.0161455 1

t = 487, loss = 1448080.125, A = [[2.0108726 1.3074353]], b = 3.8885388374328613
t = 488, loss = 1448078.875, A = [[2.0108387 1.3074018]], b = 3.8889267444610596
t = 489, loss = 1448077.875, A = [[2.0108051 1.3073684]], b = 3.889312744140625
t = 490, loss = 1448076.75, A = [[2.0107715 1.3073353]], b = 3.8896968364715576
t = 491, loss = 1448075.875, A = [[2.0107381 1.3073022]], b = 3.8900790214538574
t = 492, loss = 1448074.75, A = [[2.010705  1.3072695]], b = 3.8904595375061035
t = 493, loss = 1448073.75, A = [[2.0106719 1.3072368]], b = 3.890838146209717
t = 494, loss = 1448072.625, A = [[2.010639  1.3072042]], b = 3.8912148475646973
t = 495, loss = 1448071.75, A = [[2.0106063 1.307172 ]], b = 3.891589641571045
t = 496, loss = 1448070.625, A = [[2.0105739 1.3071398]], b = 3.8919625282287598
t = 497, loss = 1448069.5, A = [[2.0105414 1.3071078]], b = 3.892333507537842
t = 498, loss = 1448068.75, A = [[2.0105093 1.307076 ]], b = 3.892702579498291
t = 499, loss = 1448067.5, A = [[2.01047

...

t = 1537, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1538, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1539, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1540, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1541, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1542, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1543, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1544, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1545, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1546, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1547, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1548, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1549, loss = 1447991.5, A = [[2.005489  1.302109

t = 1647, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1648, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1649, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1650, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1651, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1652, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1653, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1654, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1655, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1656, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1657, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1658, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.950237274169922
t = 1659, loss = 1447991.5, A = [[2.005489  1.302109

t = 1754, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1755, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1756, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1757, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1758, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1759, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1760, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1761, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1762, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1763, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1764, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1765, loss = 1447991.625, A = [[2.005489  1.3021097]], b = 3.950237512588501
t = 1766, loss = 1447991.625

t = 1885, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1886, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1887, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1888, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1889, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1890, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1891, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1892, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1893, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1894, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1895, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1896, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1897, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.9

t = 1990, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1991, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1992, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1993, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1994, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1995, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1996, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1997, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1998, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708
t = 1999, loss = 1447991.5, A = [[2.005489  1.3021097]], b = 3.95023775100708


In [44]:
model(torch.tensor([1.1,2]))

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

## Create dataset

In [3]:
# loading relevancy data

path = 'C:/Users/User/OneDrive - NTNU/NTNU/Prosjekt oppgave NLP/dataset/CORD-19/'
file = 'relevance_data.csv'

label_data = pd.read_csv(path + file)
label_data.head()

,Unnamed: 0,topic_id,round_id,cord_uid,relevancy
0,0,1,4.5,005b2j4b,2
1,1,1,4.0,00fmeepz,1
2,2,1,0.5,010vptx3,2
3,3,1,2.5,0194oljo,1
4,4,1,4.0,021q9884,1


In [10]:
# loading topics data

path = 'C:/Users/User/OneDrive - NTNU/NTNU/Prosjekt oppgave NLP/dataset/CORD-19/'
file = 'topics.csv'

topics = pd.read_csv(path + file)
topics.head()

,query_id,query,question,narrative
0,1,coronavirus origin,what is the origin of COVID-19,seeking range of information about the SARS-Co...
1,2,coronavirus response to weather changes,how does the coronavirus respond to changes in...,seeking range of information about the SARS-Co...
2,3,coronavirus immunity,will SARS-CoV2 infected people develop immunit...,seeking studies of immunity developed due to i...
3,4,how do people die from the coronavirus,what causes death from Covid-19?,Studies looking at mechanisms of death from Co...
4,5,animal models of COVID-19,what drugs have been active against SARS-CoV o...,Papers that describe the results of testing d...


In [4]:
len(label_data)

69318

In [53]:
# available models: ['bert-base-uncased', "allenai/scibert_scivocab_uncased", "gsarti/covidbert-nli"]

def get_tesors(row, topic_field, doc_field, model):
    topic_path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/topic_embeddings/"
    doc_path = "C:/Users/User/Documents/NTNU/NLP/CORD-19/Embeddings/786/"
    doc_id = row["cord_uid"]
    topic_id = row["topic_id"]
    topic_tensor = []
    doc_tensor = []
    
    
    
    doc_embedding_file = doc_path + doc_id + ".txt"
    try:
        with open(doc_embedding_file, "rb") as fp:   # Unpickling
            doc_embedding = pickle.load(fp)
    except:
        return [[0]], [[0]]
        
    doc_tensor = doc_embedding["models"][model][doc_field]
            
    topic_embedding_file = topic_path + str(topic_id) + ".txt"
    with open(topic_embedding_file, "rb") as fp:   # Unpickling
        topic_embedding = pickle.load(fp)
        
    topic_tensor =  topic_embedding["models"][model][topic_field]
    
    return topic_tensor, doc_tensor
    

In [34]:
def join_tensors(tensor_a,tensor_b):
    n = len(tensor_a)
    m = len(tensor_b)
    c = torch.empty(n + m)
    c[0:n] = tensor_a
    c[n:(n+m)] = tensor_b
    return c

In [ ]:
def isNaN(num):
    return num != num

In [373]:
num = 20000 # max: 69318
x_dataset = torch.empty(num, 768*2)
y_dataset = torch.empty(num)
miss = 0
i = 0
for idx, row in label_data.iterrows():
    if idx < num:
        a, b = get_tesors(row, "query", "title", "allenai/scibert_scivocab_uncased")
        if len(a[0]) != 1:
            c = join_tensors(a[0],b[0])
            x_dataset[i] = c
        
            y_dataset[i] = row["relevancy"]
            i+=1
        else:
            miss +=1
print(miss)

767


In [374]:
x_dataset = x_dataset[0:(num-miss)]
y_dataset = y_dataset[0:(num-miss)]

In [375]:
dat = torch.cat((x_dataset, y_dataset.unsqueeze(1)), 1)

In [376]:
for i in dat:
    print(i[-1])

tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
t

tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
t

tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
t

tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
t

tensor(1., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
t

...

tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
t

tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
t

tensor(2., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(1., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
t

tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)
tensor(2., grad_fn=<SelectBackward>)

KeyboardInterrupt: 

In [377]:
dat[1][768*2]

tensor(1., grad_fn=<SelectBackward>)

In [378]:
print(len(y_dataset))
print(len(x_dataset))

19233
19233


In [396]:
for i in range(len(y_dataset)):
    rel = y_dataset[i]
    if rel > 0.001:
        y_dataset[i] = 1
        

In [401]:
# Hyper Parameters 
input_size = 768*2
num_classes = 2
num_epochs = 20
batch_size = 100
learning_rate = 0.001

In [402]:
# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=dat, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=dat, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [403]:
# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
train_loader

In [404]:
model = LogisticRegression(input_size, num_classes)

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Training the Model
for epoch in range(num_epochs):
    for i, (X_y) in enumerate(train_loader):
        
        y = X_y[:,768*2]
        #print(y)
        X = X_y[:,0:(768*2)].tolist()
        #print(X)
        #print(list(X.size()))
        #print(len(y))
        y = y.int()
        #print(list(y.size()))
        #print(len(y))
        y =  torch.LongTensor(y.tolist())
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(torch.tensor(X))
        loss = criterion(outputs, y)
        #print(loss)
        #print(model)
        loss.backward(retain_graph=True)

        optimizer.step()
        
        if (i+1) % 100 == 0:
            #print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
            #       % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))
            print(loss)



RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at C:\w\1\s\tmp_conda_3.7_055457\conda\conda-bld\pytorch_1565416617654\work\aten\src\THNN/generic/ClassNLLCriterion.c:94

In [388]:
# Test the Model
correct = 0
total = 0
for X_y in test_loader:
    y = X_y[:,768*2]
    y = y.int()
    y =  torch.LongTensor(y.tolist())
    #print(y)
    X = X_y[:,0:(768*2)].tolist()
    X = torch.tensor(X)
    outputs = model(X)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == y).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the model on the 10000 test images: 65 %


In [395]:
y_dataset.int().tolist().count(0)/len(y_dataset)

0.6435813445640306

In [387]:
for i, x in enumerate (x_dataset):
    if i < 10:
        print(i)
        print(len(x))
        p = model(x)
        print(p)
        print(y_dataset[i])

0
1536
tensor([ 0.8123, -0.0831, -0.3060], grad_fn=<AddBackward0>)
tensor(2.)
1
1536
tensor([ 0.7913, -0.1579, -0.4823], grad_fn=<AddBackward0>)
tensor(1.)
2
1536
tensor([ 0.8824,  0.0047, -0.3716], grad_fn=<AddBackward0>)
tensor(2.)
3
1536
tensor([ 1.1005, -0.1135, -0.5846], grad_fn=<AddBackward0>)
tensor(1.)
4
1536
tensor([ 1.0453, -0.2018, -0.6070], grad_fn=<AddBackward0>)
tensor(1.)
5
1536
tensor([ 0.9314, -0.1476, -0.4723], grad_fn=<AddBackward0>)
tensor(1.)
6
1536
tensor([ 0.5052, -0.0994,  0.0588], grad_fn=<AddBackward0>)
tensor(0.)
7
1536
tensor([ 1.2799, -0.2171, -0.8482], grad_fn=<AddBackward0>)
tensor(0.)
8
1536
tensor([ 0.7166, -0.0851, -0.4365], grad_fn=<AddBackward0>)
tensor(0.)
9
1536
tensor([ 1.0933, -0.2860, -0.7192], grad_fn=<AddBackward0>)
tensor(0.)


In [303]:


import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable


In [304]:

# Hyper Parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [305]:
# MNIST Dataset (Images and Labels)
train_dataset = dsets.MNIST(root='.', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=False)

test_dataset = dsets.MNIST(root='.', 
                           train=False, 
                           transform=transforms.ToTensor())

In [306]:
img_tensor, label = train_dataset[0]
print(img_tensor)
print(label)

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,

In [307]:
# Dataset Loader (Input Pipline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [308]:


# Model
class LogisticRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_size, num_classes)
    
    def forward(self, x):
        out = self.linear(x)
        return out



In [309]:
for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        print(list(dat.size()))
        print(images)
        print(labels)
        print(i)

[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 1, 2, 1, 8, 0, 9, 7, 4, 3, 1, 8, 8, 0, 4, 8, 4, 3, 0, 5, 6, 7, 9, 5,
        4, 9, 6, 5, 3, 4, 2, 8, 2, 3, 6, 3, 4, 9, 1, 5, 1, 9, 4, 4, 1, 8, 7, 0,
        4, 6, 4, 6, 1, 5, 9, 0, 5, 4, 5, 0, 8, 1, 9, 2, 3, 8, 6, 6, 4, 1, 9, 5,
        4, 4, 3, 6, 0, 2, 7, 2, 8, 2, 0, 6, 1, 7, 5, 6, 4, 3, 7, 9, 8, 9, 0, 0,
        9, 4, 8, 3])
0
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([0, 9, 3, 1, 9, 5, 1, 5, 4, 4, 9, 6, 0, 7, 7, 3, 2, 6, 6, 1, 4, 9, 6, 1,
        4, 7, 7, 3, 6, 5, 4, 2, 8, 7, 7, 7, 7, 

        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([4, 7, 0, 1, 5, 0, 0, 8, 9, 8, 7, 9, 6, 1, 8, 2, 3, 9, 5, 8, 8, 5, 6, 7,
        2, 5, 2, 2, 1, 0, 7, 6, 1, 3, 1, 1, 4, 0, 8, 9, 0, 3, 6, 0, 7, 5, 3, 6,
        0, 9, 7, 8, 3, 7, 5, 5, 8, 6, 9, 3, 0, 1, 6, 7, 2, 9, 1, 2, 5, 7, 3, 9,
        7, 3, 7, 8, 4, 7, 5, 1, 1, 3, 7, 7, 9, 2, 1, 0, 1, 7, 8, 6, 3, 2, 0, 7,
        4, 6, 6, 4])
17
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([7, 5, 2, 9, 4, 2, 2, 1, 7, 6, 9, 2, 4, 1, 1, 5, 0, 0, 1, 7, 6, 3, 8, 4,
        2, 1, 1, 5, 2, 6, 6, 7, 9, 8, 5, 0, 3, 6, 0, 6, 1, 3, 6, 5, 6, 3, 6, 6,
        5, 7, 0, 3, 2, 7, 8, 2, 4, 7, 4, 1, 9, 5, 2, 3, 3, 2, 9, 5, 0, 3, 7, 6,
        0, 9, 5, 9, 6, 0, 2, 4, 9, 7, 0, 8, 8, 6, 4, 6, 1, 8, 8, 5, 2, 8, 0, 6,
        0, 1, 4, 5])
18
[32, 1

[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([2, 6, 5, 6, 0, 7, 1, 0, 6, 1, 0, 4, 1, 1, 6, 8, 5, 3, 8, 6, 4, 7, 1, 7,
        3, 3, 4, 0, 5, 5, 3, 1, 3, 8, 8, 5, 3, 1, 0, 3, 6, 0, 1, 8, 0, 9, 6, 7,
        1, 9, 7, 2, 0, 2, 6, 4, 1, 8, 1, 7, 8, 7, 1, 9, 4, 6, 9, 8, 0, 3, 1, 9,
        7, 3, 0, 3, 4, 2, 6, 6, 4, 3, 7, 9, 3, 4, 5, 6, 3, 0, 6, 7, 9, 4, 0, 2,
        2, 5, 6, 6])
35
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([0, 3, 1, 6, 5, 4, 9, 6, 3, 7, 5, 8, 1, 6, 9, 7, 4, 5, 0, 6, 7, 9, 9, 2,
        2, 6, 6, 9, 7, 8, 1, 9, 6, 5, 4, 0, 0,

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([7, 3, 8, 0, 4, 3, 5, 2, 6, 0, 8, 5, 5, 9, 3, 0, 2, 8, 8, 6, 1, 7, 5, 1,
        9, 9, 8, 4, 8, 2, 4, 6, 1, 7, 5, 9, 4, 5, 9, 1, 2, 9, 5, 0, 6, 1, 9, 4,
        8, 1, 1, 3, 0, 9, 6, 7, 0, 0, 0, 6, 5, 3, 1, 6, 0, 9, 7, 3, 8, 9, 8, 2,
        5, 1, 5, 6, 3, 6, 4, 6, 4, 1, 3, 6, 5, 3, 0, 0, 4, 0, 8, 8, 5, 8, 6, 6,
        9, 2, 7, 5])
54
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([1, 3, 5, 4, 6, 7, 7, 0, 7, 2, 1, 4, 9, 0, 4, 2, 2, 3, 7, 4, 2, 0, 5, 0,
        5, 4, 1, 7, 1, 1, 7, 6, 8, 5, 6, 8, 0, 0, 8, 3, 3

[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([4, 0, 2, 6, 5, 3, 6, 7, 5, 7, 8, 5, 1, 4, 1, 5, 3, 6, 1, 0, 2, 9, 8, 4,
        9, 6, 5, 9, 8, 2, 7, 6, 9, 6, 2, 5, 4, 8, 0, 2, 4, 0, 7, 6, 0, 0, 7, 8,
        6, 5, 1, 1, 3, 2, 5, 2, 5, 4, 2, 9, 6, 5, 1, 2, 5, 1, 8, 8, 4, 3, 1, 6,
        2, 8, 6, 3, 5, 9, 5, 2, 1, 3, 9, 9, 1, 4, 4, 9, 9, 0, 2, 4, 1, 3, 1, 9,
        7, 1, 8, 8])
73
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([4, 4, 1, 2, 5, 8, 3, 4, 4, 6, 8, 5, 3, 8, 6, 6, 8, 9, 1, 2, 8, 1, 8, 4,
        8, 9, 8, 3, 0, 8, 8, 2, 7, 4, 3, 2, 1,

...

[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([1, 2, 9, 6, 3, 9, 5, 8, 5, 7, 0, 8, 4, 2, 2, 1, 8, 6, 9, 1, 2, 5, 3, 7,
        4, 2, 6, 1, 9, 2, 6, 9, 2, 8, 5, 1, 8, 9, 9, 9, 5, 1, 0, 6, 2, 6, 2, 0,
        7, 9, 2, 4, 8, 6, 4, 4, 7, 0, 0, 3, 6, 0, 9, 0, 3, 6, 3, 5, 2, 8, 4, 1,
        4, 2, 4, 1, 6, 8, 6, 7, 5, 0, 2, 1, 5, 6, 5, 3, 7, 8, 3, 8, 6, 3, 9, 7,
        9, 1, 5, 4])
498
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([9, 4, 7, 7, 4, 1, 4, 5, 2, 0, 0, 1, 7, 8, 9, 3, 4, 1, 3, 7, 2, 1, 1, 4,
        2, 5, 9, 6, 1, 5, 3, 2, 7, 5, 5, 3, 9

[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([6, 3, 0, 8, 0, 2, 9, 5, 1, 9, 7, 5, 8, 2, 6, 4, 4, 0, 0, 8, 1, 3, 2, 8,
        2, 5, 9, 1, 5, 8, 5, 8, 2, 6, 0, 9, 6, 8, 5, 1, 0, 0, 2, 2, 8, 8, 6, 4,
        6, 0, 7, 3, 4, 9, 6, 0, 9, 5, 7, 5, 3, 5, 7, 8, 6, 0, 4, 2, 3, 7, 7, 3,
        4, 1, 7, 5, 3, 3, 9, 6, 0, 0, 8, 6, 8, 8, 5, 0, 8, 9, 4, 6, 7, 3, 0, 5,
        1, 1, 0, 0])
520
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([4, 7, 0, 3, 7, 8, 1, 5, 8, 5, 4, 3, 1, 3, 5, 5, 1, 4, 6, 2, 8, 8, 8, 7,
        2, 6, 5, 1, 2, 1, 8, 7, 9, 9, 2, 6, 0

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([1, 3, 4, 4, 0, 3, 4, 9, 7, 4, 0, 3, 3, 0, 7, 1, 2, 5, 5, 3, 9, 0, 1, 9,
        1, 5, 4, 9, 3, 3, 8, 1, 8, 9, 9, 7, 7, 1, 0, 8, 3, 5, 3, 4, 3, 0, 6, 0,
        4, 4, 8, 8, 7, 5, 8, 4, 7, 6, 5, 7, 1, 3, 2, 8, 9, 8, 0, 2, 3, 2, 2, 7,
        4, 8, 5, 1, 3, 3, 1, 8, 7, 7, 5, 0, 5, 3, 3, 8, 8, 1, 9, 5, 2, 0, 0, 2,
        8, 8, 9, 2])
543
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 6, 6, 5, 3, 3, 3, 8, 3, 1, 0, 0, 4, 7, 0, 5, 7, 2, 4, 8, 3, 0, 0, 7,
        2, 9, 9, 6, 4, 7, 8, 6, 0, 8, 0, 6, 9, 6, 7, 0, 

        8, 1, 0, 1])
564
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([4, 2, 1, 2, 3, 3, 1, 4, 5, 9, 3, 9, 4, 5, 1, 8, 7, 1, 3, 0, 8, 3, 7, 5,
        0, 4, 5, 0, 3, 2, 4, 2, 7, 8, 9, 2, 2, 6, 3, 4, 1, 2, 8, 6, 0, 1, 4, 9,
        4, 5, 2, 5, 7, 6, 2, 0, 6, 0, 8, 5, 7, 4, 1, 4, 3, 4, 5, 0, 3, 7, 1, 1,
        5, 8, 7, 5, 8, 5, 1, 1, 9, 7, 6, 2, 4, 5, 4, 2, 5, 3, 5, 0, 2, 5, 9, 9,
        2, 4, 4, 4])
565
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([0, 3, 3, 3, 3, 6, 5, 1, 0, 3, 0, 7, 7, 5, 9, 9, 3, 9, 2, 2, 3, 4, 2, 6,
        6, 9, 9, 2, 

[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([4, 5, 6, 4, 0, 8, 5, 4, 5, 4, 7, 7, 4, 3, 2, 6, 8, 5, 7, 8, 4, 3, 0, 8,
        8, 4, 9, 3, 1, 4, 6, 9, 2, 3, 9, 3, 3, 0, 4, 1, 0, 2, 4, 3, 4, 7, 2, 0,
        3, 1, 2, 7, 4, 0, 8, 6, 1, 8, 4, 4, 9, 6, 0, 0, 2, 3, 1, 0, 9, 4, 0, 4,
        9, 6, 4, 8, 8, 5, 8, 4, 0, 9, 1, 3, 2, 1, 0, 8, 4, 2, 3, 7, 4, 1, 6, 7,
        1, 0, 0, 9])
587
[32, 1537]
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([2, 0, 1, 7, 4, 6, 4, 0, 0, 3, 9, 9, 2, 9, 7, 0, 6, 0, 1, 7, 1, 7, 6, 3,
        4, 3, 3, 1, 3, 3, 5, 3, 7, 7, 2, 8, 9

In [312]:
model = LogisticRegression(input_size, num_classes)

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

# Training the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        #print(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(images[0])
            print(list(images.size()))
            print(list(labels.size()))
            #print ('Epoch: [%d/%d], Step: [%d/%d], Loss: %.4f' 
             #      % (epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

# Test the Model
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1255, 1.0000,
        0.9961, 0.3020, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 

KeyboardInterrupt: 